In [1]:
import foodmood_lists as FM
import functools
import pandas
from pandas.io import sql
import sqlalchemy
import pymysql
import numpy

In [2]:
recipe_data = pandas.read_csv("RAW_recipes.csv")

In [3]:
def listify(tag_string):
    return [tag.strip("'") for tag in tag_string.strip('][').split(", ")]

In [4]:
tag_lists = recipe_data['tags'].apply(listify)
unique_tags = functools.reduce(lambda a,b: set(a).union(set(b)), tag_lists.to_list())

In [5]:
recipe_data.ingredients = recipe_data.ingredients.apply(listify)
recipe_data.tags = recipe_data.tags.apply(listify)
recipe_data.nutrition = recipe_data.nutrition.apply(listify)
recipe_data.steps = recipe_data.steps.apply(listify)

In [6]:
units = ['tsp', 'tbsp', 'oz', 'cup', 'pint', 'quart', 'gallon', 'mL', 'L', 'dL', 'lb', 'mg', 'g', 'kg']

In [7]:
ingredient_table = recipe_data[['id', 'ingredients']].explode('ingredients')
ingredient_table['quantity'] = numpy.random.randint(1,10, ingredient_table.shape[0])
rand_units = [units[i%14] for i in range(ingredient_table.shape[0])]
ingredient_table['units'] = rand_units
ingredient_table.columns = ['recipeId', 'foodName', 'quantity', 'unit']

In [8]:
ingredient_table

,recipeId,foodName,quantity,unit
0,137739,winter squash,5,tsp
0,137739,mexican seasoning,8,tbsp
0,137739,mixed spice,8,oz
0,137739,honey,6,cup
0,137739,butter,5,pint
...,...,...,...,...
231636,298509,eggs,9,pint
231636,298509,flour,1,quart
231636,298509,cream of tartar,9,gallon
231636,298509,baking soda,3,mL


In [9]:
tag_table = recipe_data[['id', 'tags']].explode('tags')
tag_table = tag_table[~tag_table.tags.isin(FM.labels) & (~tag_table.tags.isin(FM.cuisines))]
tag_table.columns = ['recipeId', 'tagName']

In [10]:
tag_table

,recipeId,tagName
0,137739,60-minutes-or-less
0,137739,vegetables
0,137739,easy
0,137739,fall
0,137739,vegetarian
...,...,...
231636,298509,desserts
231636,298509,kid-friendly
231636,298509,cookies-and-brownies
231636,298509,dietary


In [11]:
recipe_table = recipe_data[['id', 'name', 'minutes', 'n_steps', 'tags', 'n_ingredients', 'contributor_id']]
recipe_table.columns = ['recipeId', 'recipeName', 'cookTime', 'difficulty', 'cuisine', 'servings', 'authorName']

In [12]:
def find_tags(tags, tag_set):
    tagged = list(set(tags).intersection(tag_set))
    if tagged:
        return tagged[-1]
    else:
        return None

In [13]:
recipe_table.cuisine = recipe_table.cuisine.apply(find_tags, args=(FM.cuisines,))

In [14]:
names = pandas.read_csv('StateNames.csv').Name

In [15]:
def id_toName(authorID):
    return names[authorID%names.size]

In [16]:
recipe_table.authorName = recipe_table.authorName.apply(id_toName)

In [17]:
recipe_table['instructionsLink'] = "https://www.food.com/recipe/" + recipe_table.recipeName.str.replace('\s+', '-').str.cat(recipe_table.recipeId.astype('str'), '-')

In [18]:
recipe_table = recipe_table.dropna()

In [19]:
recipe_table.recipeName = recipe_table.recipeName.str.replace('\s+', ' ').str.title()
recipe_table.cuisine = recipe_table.cuisine.str.title()

In [20]:
recipe_table

,recipeId,recipeName,cookTime,difficulty,cuisine,servings,authorName,instructionsLink
0,137739,Arriba Baked Winter Squash Mexican Style,55,11,North-American,7,Deanna,https://www.food.com/recipe/arriba-baked-winte...
1,31490,A Bit Different Breakfast Pizza,30,9,American,6,Brody,https://www.food.com/recipe/a-bit-different-br...
4,44061,Amish Tomato Ketchup For Canning,190,5,North-American,8,Hettie,https://www.food.com/recipe/amish-tomato-ketch...
5,5289,Apple A Day Milk Shake,0,4,American,4,Charlotte,https://www.food.com/recipe/apple-a-day-milk-s...
6,25274,Aww Marinated Olives,15,4,North-American,9,Luke,https://www.food.com/recipe/aww-marinated-oliv...
...,...,...,...,...,...,...,...,...
231626,61748,Zwiebelsuppe Clear German Onion Soup,35,10,European,11,Lilla,https://www.food.com/recipe/zwiebelsuppe-clear...
231627,410985,Zwiebelwhe Swiss Onion Tart,165,38,European,13,Susie,https://www.food.com/recipe/zwiebelwhe-swiss-o...
231630,357451,Zydeco Sauce,15,3,American,6,Sharae,https://www.food.com/recipe/zydeco-sauce-357451
231632,486161,Zydeco Soup,60,7,Southern-United-States,22,Casey,https://www.food.com/recipe/zydeco-soup-486161


In [21]:
keyFile = open("APIkey.txt", "r")
key = keyFile.read()
keyFile.close()

In [22]:
from google_images_search import GoogleImagesSearch
foodImage = GoogleImagesSearch(key, '017970115376021606908:q3caxr88ame')

In [24]:
imageLinks = []
recipe_names = recipe_table.recipeName.to_list()

In [25]:

for recipe_name in recipe_names:

    _searchParams = {
        'q': recipe_name,
        'num': 1,
        'fileType': 'jpg|png'
    }
    try:
        foodImage.search(search_params=_searchParams)

        if foodImage.results():
            imageLinks.append(foodImage.results()[0].url)
        else:
            imageLinks.append(None)
    except:
        imageLinks.append(None)


In [38]:
recipe_table['imageUrl'] = imageLinks
recipe_table = recipe_table.dropna()

,recipeId,recipeName,cookTime,difficulty,cuisine,servings,authorName,instructionsLink,imageUrl
0,137739,arriba baked winter squash mexican style,55,11,north-american,7,Deanna,https://www.food.com/recipe/arriba-baked-winte...,https://img.sndimg.com/food/image/upload/c_thu...
1,31490,a bit different breakfast pizza,30,9,american,6,Brody,https://www.food.com/recipe/a-bit-different-br...,https://img.sndimg.com/food/image/upload/c_thu...
4,44061,amish tomato ketchup for canning,190,5,heirloom-historical,8,Hettie,https://www.food.com/recipe/amish-tomato-ketch...,https://growagoodlife.com/wp-content/uploads/2...
5,5289,apple a day milk shake,0,4,american,4,Charlotte,https://www.food.com/recipe/apple-a-day-milk-s...,https://images.happycow.net/venues/1024/85/88/...
6,25274,aww marinated olives,15,4,north-american,9,Luke,https://www.food.com/recipe/aww-marinated-oliv...,https://img.sndimg.com/food/image/upload/c_thu...
7,67888,backyard style barbecued ribs,120,10,north-american,22,Claire,https://www.food.com/recipe/backyard-style-bar...,https://assets.wsimgs.com/wsimgs/ab/images/dp/...
11,42198,better than sex strawberries,1460,8,comfort-food,7,Effie,https://www.food.com/recipe/better-than-sex-st...,https://pbs.twimg.com/profile_images/955857200...
12,67547,better then bush s baked beans,2970,9,american,13,Dasia,https://www.food.com/recipe/better-then-bush-s...,https://img.sndimg.com/food/image/upload/c_thu...
16,43026,chile rellenos,45,9,american,5,Anita,https://www.food.com/recipe/chile-rellenos-43026,https://www.isabeleats.com/wp-content/uploads/...
18,8559,chinese chop suey,70,8,american,7,Roberta,https://www.food.com/recipe/chinese-chop-suey-...,https://img.sndimg.com/food/image/upload/w_555...


**Run in the shell** 

./cloud_sql_proxy -instances=cs348s2020:us-central1:cs348-1=tcp:3306

In [34]:
mySql_config = {
    "pool_size": 5,
    "max_overflow": 2,
    "pool_timeout": 30,
    "pool_recycle": 1800,
}
mySql_engine = sqlalchemy.create_engine(
                sqlalchemy.engine.url.URL(
                    drivername="mysql+pymysql",
                    username='root',
                    password='cs348',
                    host='127.0.0.1',
                    port='3306',
                    database='db_1',
                ),
                **mySql_config
            )

In [35]:
from sqlalchemy.types import Integer
from sqlalchemy.types import String

mySql_connection = mySql_engine.connect()
try:
    recipe_frame = recipe_table.to_sql('recipe', mySql_connection, if_exists='replace', 
                        index=False, chunksize=50, 
                        dtype={'recipeId': Integer(), 'recipeName': String(60), 'cookTime': Integer(), 
                                'difficulty': Integer(), 'cuisine': String(40), 'servings': Integer(),      
                                'authorName': String(40), 'instructionsLink': String(225), 'imageUrl': String(225)})

    ingredient_frame = ingredient_table.to_sql('ingredient', mySql_connection, if_exists='replace',
                        index=False, chunksize=50,
                        dtype={'recipeId': Integer(), 'foodName': String(255), 'qantity': Integer(), 
                                'unit': String(20)})

    tag_frame = tag_table.to_sql('tags', mySql_connection, if_exists='replace',
                        index=False, chunksize=50,
                        dtype={'recipeId': String(24), 'tagName': String(40)})

except ValueError as vx:
    print(vx)

except Exception as ex:   
    print(ex)

else:
    print("Tables created successfully");   

finally:
    mySql_connection.close()

Tables created successfully
